In [1]:
import pandas as pd
from freesurfer_stats import CorticalParcellationStats
import glob
import re
import itertools
import numpy as np
import pyls
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
import pickle
from sklearn import preprocessing
from scipy.stats import zscore
from nilearn import input_data
from nilearn import plotting

/opt/miniconda3/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
sns.set_context("notebook", font_scale=1.7)
sns.set_style("whitegrid")

## PLS ANALYSIS

In [3]:
all_volumes_file = 'data/all_volumes.csv'
all_volumes = pd.read_csv(all_volumes_file)
all_volumes.drop(columns=all_volumes.columns[0], axis=1, inplace=True)
#all_volumes.columns.to_list()

In [5]:
# Selección de estructuras, no se inlcluye el 5to ventrículo
columns = list(range(0,15)) + list(range(21,54)) + list(range(61,467))
#columns = list(range(0,4)) + list(range(21,54)) + list(range(61,467))
selected_values = all_volumes.iloc[:,columns]
#list(selected_values.columns)

## NORMALIZATION BY ETIV

In [ ]:
selected_values.iloc[:,3:] = selected_values.iloc[:,3:].div(selected_values["Estimated Total Intracranial Volume"], axis=0)
selected_values

In [ ]:
all_data = selected_values
#all_data[all_data['subject']=='TABP079']

In [ ]:
# Remover columna de ETIV
all_data = all_data.drop(['Estimated Total Intracranial Volume'],axis=1)
#all_data

## Clinical data

In [ ]:
clinical_file = 'data/clinical_data_prisma_VF.xlsx'
clinical_data = pd.read_excel(clinical_file,'master')
clinical_data

In [ ]:
clinical_data = clinical_data.iloc[:, [9,0,2,3,4,5,6,7,8,15]]
clinical_data.rename(columns = {'id_RM':'subject'}, inplace = True)
#clinical_data

In [ ]:
clinical_data

## Edit file

In [ ]:
pd.set_option('display.max_rows', None)

#add same treatment in all times
l=['Group','time','edad','gen','esc','dx','psc', 'Gravedad'] # column different from each row
clinical_data_f = pd.concat([clinical_data.drop(l,1).groupby('subject').transform('first'),clinical_data[l],clinical_data[['subject']]],axis=1).reindex(columns=clinical_data.columns)

clinical_data_f['project'] = 'prisma'+ clinical_data_f.time.astype(str)

clinical_data_f.loc[clinical_data_f.Group == 'CTR', 'project'] = "controles"
# Agregar condición
#clinical_data_f['c_group'] = clinical_data_f['Group'] + '-' + clinical_data_f['project'] +'-' +'treatment' + clinical_data_f['treatment'].astype(str)
clinical_data_f['c_group'] = clinical_data_f['Group'] + '-' + clinical_data_f['project'] 

In [ ]:
clinical_data_f

## Merge both files

In [ ]:
clinical_image_df= pd.merge(all_data, clinical_data_f[['subject','project','c_group','treatment','psc','dx','Gravedad','edad']], how='left', on=['subject', 'project'])

#Remove data for TAB and diagnosis 2
clinical_image_df.drop(clinical_image_df.index[(clinical_image_df.subject.str.contains('TAB'))&(clinical_image_df.dx==2)],inplace=True)

# Remove subject with NAs in volumetric nan
#clinical_image_df.drop(clinical_image_df.index[clinical_image_df.subject=="TABP079"],inplace=True)


In [ ]:
#clinical_image_df

In [ ]:
clinical_image_df[clinical_image_df['subject']=='TABP079']
# Remove subject with NAs in volumetric and clinical data
clinical_image_df.drop(clinical_image_df.index[clinical_image_df.subject=="TABP079"],inplace=True)

### NORMALIZE BY GROUP

## Standardscaler

In [ ]:
# Standardize features by removing the mean and scaling to unit variance
clinical_image_df_n_CTR= clinical_image_df[clinical_image_df.grupo=='CTR']
clinical_image_df_n_EQF= clinical_image_df[clinical_image_df.grupo=='EQF']
clinical_image_df_n_TAB= clinical_image_df[clinical_image_df.grupo=='TAB']
clinical_image_df_n_CTR.iloc[:,3:-6] = preprocessing.StandardScaler().fit_transform(clinical_image_df_n_CTR.iloc[:,3:-6])
clinical_image_df_n_TAB.iloc[:,3:-6] = preprocessing.StandardScaler().fit_transform(clinical_image_df_n_TAB.iloc[:,3:-6])
clinical_image_df_n_EQF.iloc[:,3:-6] = preprocessing.StandardScaler().fit_transform(clinical_image_df_n_EQF.iloc[:,3:-6])


## Z-score by subject

In [ ]:
# Normalización z-score por fila (sujeto)
clinical_image_z = clinical_image_df
clinical_image_z.iloc[:,3:-6] = clinical_image_z.iloc[:,3:-6].apply(zscore, axis=1) 

In [ ]:
clinical_image_n = pd.concat([clinical_image_df_n_CTR,clinical_image_df_n_TAB,clinical_image_df_n_EQF])

In [ ]:
clinical_image_i = clinical_image_z # Selección normalización por z-score
#clinical_image_i = clinical_image_n # Selección normalización por grupo
clinical_image_EQF = clinical_image_i[clinical_image_i.grupo!='TAB'] # Toma EQF y CTRL
clinical_image_TAB = clinical_image_i[clinical_image_i.grupo!='EQF'] # Toma TAB y CTRL

In [ ]:
# Formación datos EQF y TAB --> PSI
clinical_image_i['grupo-p'] = ''
clinical_image_i.loc[clinical_image_i.grupo.str.contains('TAB'), 'grupo-p'] = 'PSI'
clinical_image_i.loc[clinical_image_i.grupo.str.contains('EQF'), 'grupo-p'] = 'PSI'
clinical_image_i.loc[clinical_image_i.grupo.str.contains('CTR'), 'grupo-p'] = 'CTR'
# Modificar condición
#clinical_image_i['c_group'] = clinical_image_i['grupo'] + '-' + clinical_image_i['project'] +'-' +'treatment' + clinical_image_i['treatment'].astype(str)
clinical_image_i['c_group'] = clinical_image_i['grupo-p'] + '-' + clinical_image_i['project'] + '-' + clinical_image_i['grupo']


In [ ]:
all_data_sorted_EQF = clinical_image_EQF.sort_values('c_group', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

count_groups_EQF = all_data_sorted_EQF.groupby(['c_group']).count()
groups_dict_EQF = count_groups_EQF['subject'].to_dict()

list_len_EQF = list(groups_dict_EQF.values())
groups_EQF = list(groups_dict_EQF.keys())
groups_dict_EQF

In [ ]:
all_data_sorted_EQF = all_data_sorted_EQF.dropna(subset=['c_group'])
#all_data_sorted_EQF

In [ ]:
count_groups_EQF = all_data_sorted_EQF.groupby(['c_group']).count()
groups_dict_EQF = count_groups_EQF['subject'].to_dict()

list_len_EQF = list(groups_dict_EQF.values())
groups_EQF = list(groups_dict_EQF.keys())
groups_dict_EQF

In [ ]:
all_data_sorted_TAB = clinical_image_TAB.sort_values('c_group', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

count_groups_TAB = all_data_sorted_TAB.groupby(['c_group']).count()
groups_dict_TAB = count_groups_TAB['subject'].to_dict()

list_len_TAB = list(groups_dict_TAB.values())
groups_TAB = list(groups_dict_TAB.keys())
groups_dict_TAB

In [ ]:
all_data_sorted_TAB = all_data_sorted_TAB.dropna(subset=['c_group'])
#all_data_sorted_TAB

In [ ]:
count_groups_TAB = all_data_sorted_TAB.groupby(['c_group']).count()
groups_dict_TAB = count_groups_TAB['subject'].to_dict()

list_len_TAB = list(groups_dict_TAB.values())
groups_TAB = list(groups_dict_TAB.keys())
groups_dict_TAB

In [ ]:
all_data_sorted_PSI = clinical_image_i.sort_values('c_group', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

count_groups_PSI = all_data_sorted_PSI.groupby(['c_group']).count()
groups_dict_PSI = count_groups_PSI['subject'].to_dict()

list_len_PSI = list(groups_dict_PSI.values())
groups_PSI = list(groups_dict_PSI.keys())
groups_dict_PSI

In [ ]:
all_data_sorted_PSI = all_data_sorted_PSI.dropna(subset=['c_group'])
count_groups_PSI= all_data_sorted_PSI.groupby(['c_group']).count()
groups_dict_PSI = count_groups_PSI['subject'].to_dict()

list_len_PSI = list(groups_dict_PSI.values())
groups_PSI = list(groups_dict_PSI.keys())
groups_dict_PSI

## PLS WITH CLINICAL GROUPS

In [ ]:
X_EQF = all_data_sorted_EQF.iloc[:,3:-6].values
X_TAB = all_data_sorted_TAB.iloc[:,3:-6].values
X_PSI = all_data_sorted_PSI.iloc[:,3:-7].values

In [ ]:
random_state=42
pls_EQF = pyls.meancentered_pls(X_EQF, groups=list_len_EQF, seed=42, mean_centering=1, n_proc='max')

In [ ]:
#with open('pls_V_EQF_n.pkl', 'wb') as f:
with open('pls_EQF.pkl', 'wb') as f:
    pickle.dump(pls_EQF, f)

In [ ]:
with open("pls_EQF.pkl", "rb") as input_file:
    pls_EQF = pickle.load(input_file)

In [ ]:
pls_TAB = pyls.meancentered_pls(X_TAB, groups=list_len_TAB, mean_centering=1, seed=42, n_proc='max')

In [ ]:
with open('pls_TAB.pkl', 'wb') as f:
    pickle.dump(pls_TAB, f)

In [ ]:
with open("pls_TAB.pkl", "rb") as input_file:
    pls_TAB = pickle.load(input_file)

In [ ]:
pls_PSI = pyls.meancentered_pls(X_PSI, groups=list_len_PSI, seed=42, mean_centering=1, n_proc='max')

In [ ]:
# Significancia PLS grupos EQF y CTRL, tiempo y tratamiento
pvals_EQF = pls_EQF.permres['pvals']
x_weights_EQF = pls_EQF.bootres.x_weights_normed 
y_weights_EQF = pls_EQF.y_weights
pvals_EQF

In [ ]:
# Significancia grupos TAB y CTRL, tiempo y tratamiento
pvals_TAB = pls_TAB.permres['pvals']
x_weights_TAB = pls_TAB.bootres.x_weights_normed
y_weights_TAB = pls_TAB.y_weights
pvals_TAB

In [ ]:
# Significancia grupos EQF+TAB y CTRL, tiempo y tratamiento
pvals_PSI = pls_PSI.permres['pvals']
x_weights_PSI = pls_PSI.bootres.x_weights_normed
y_weights_PSI = pls_PSI.y_weights
pvals_PSI

In [ ]:
def get_pos(n_col, init=-0.3, inte=0.6):
    pos = []
    for i in range(n_col):
        x = init 
        init = round(x + inte, 1)
        pos.append(x)
    return pos

def errplot(x,y, yerr, **kwargs):
    data = kwargs.pop("data")
    plt.errorbar(x=x,y=data[y],yerr=data[yerr],fmt='none', c= 'k')

In [ ]:
# Incluyendo tratamiento
#y_loadings_df_EQF = pd.DataFrame({'1': y_weights_EQF[:,0], '2': y_weights_EQF[:,1], 'condition': groups_EQF,
#                                  'tratamiento': [group.split("-")[2] for group in groups_EQF] , 'time': [group.split("-")[1] for group in groups_EQF]})

#y_loadings_m_EQF = pd.melt(y_loadings_df_EQF,id_vars=['condition','tratamiento','time'],value_name='weight', var_name = 'salvar_id')

# Sin tratamiento
y_loadings_df_EQF = pd.DataFrame({'1': y_weights_EQF[:,0], '2': y_weights_EQF[:,1], 'condition': groups_EQF,
                                  'time': [group.split("-")[1] for group in groups_EQF]})

y_loadings_m_EQF = pd.melt(y_loadings_df_EQF,id_vars=['condition','time'],value_name='weight', var_name = 'salvar_id')

y_loadings_m_EQF['ci_l'] = np.concatenate([pls_EQF.bootres.contrast_ci[:,0,0], pls_EQF.bootres.contrast_ci[:,1,0]])
y_loadings_m_EQF['ci_u'] = np.concatenate([pls_EQF.bootres.contrast_ci[:,0,1], pls_EQF.bootres.contrast_ci[:,1,1]])

y_loadings_m_EQF['ci_l_p'] = y_loadings_m_EQF['weight'] - y_loadings_m_EQF['ci_l']
y_loadings_m_EQF['ci_u_p'] = y_loadings_m_EQF['weight'] - y_loadings_m_EQF['ci_u']
y_loadings_m_EQF['err'] = (y_loadings_m_EQF['ci_u'] - y_loadings_m_EQF['ci_l'])/2

In [ ]:
y_loadings_m_EQF

In [ ]:
#y_loadings_df_TAB = pd.DataFrame({'1': y_weights_TAB[:,0], '2': y_weights_TAB[:,1], 'condition': groups_TAB,
#                                 'tratamiento': [group.split("-")[2] for group in groups_TAB] , 'time': [group.split("-")[1] for group in groups_TAB]})
#y_loadings_m_TAB = pd.melt(y_loadings_df_TAB,id_vars=['condition','tratamiento','time'],value_name='weight', var_name = 'salvar_id')

y_loadings_df_TAB = pd.DataFrame({'1': y_weights_TAB[:,0], '2': y_weights_TAB[:,1], 'condition': groups_TAB,
                                  'time': [group.split("-")[1] for group in groups_TAB]})
y_loadings_m_TAB = pd.melt(y_loadings_df_TAB,id_vars=['condition','time'],value_name='weight', var_name = 'salvar_id')

y_loadings_m_TAB['ci_l'] = np.concatenate([pls_TAB.bootres.contrast_ci[:,0,0], pls_TAB.bootres.contrast_ci[:,1,0]])
y_loadings_m_TAB['ci_u'] = np.concatenate([pls_TAB.bootres.contrast_ci[:,0,1], pls_TAB.bootres.contrast_ci[:,1,1]])

y_loadings_m_TAB['ci_l_p'] = y_loadings_m_TAB['weight'] - y_loadings_m_TAB['ci_l']
y_loadings_m_TAB['ci_u_p'] = y_loadings_m_TAB['weight'] - y_loadings_m_TAB['ci_u']
y_loadings_m_TAB['err'] = (y_loadings_m_TAB['ci_u'] - y_loadings_m_TAB['ci_l'])/2

In [ ]:
y_loadings_df_PSI = pd.DataFrame({'1': y_weights_PSI[:,0], '2': y_weights_PSI[:,1], 'condition': groups_PSI,
                                  'enfermedad': [group.split("-")[2] for group in groups_PSI] , 'time': [group.split("-")[1] for group in groups_PSI]})

y_loadings_m_PSI = pd.melt(y_loadings_df_PSI,id_vars=['condition','enfermedad','time'],value_name='weight', var_name = 'salvar_id')

#y_loadings_df_PSI = pd.DataFrame({'1': y_weights_PSI[:,0], '2': y_weights_PSI[:,1], 'condition': groups_PSI,
#                                  'time': [group.split("-")[1] for group in groups_PSI]})

#y_loadings_m_PSI = pd.melt(y_loadings_df_PSI,id_vars=['condition','time'],value_name='weight', var_name = 'salvar_id')

y_loadings_m_PSI['ci_l'] = np.concatenate([pls_PSI.bootres.contrast_ci[:,0,0], pls_PSI.bootres.contrast_ci[:,1,0]])
y_loadings_m_PSI['ci_u'] = np.concatenate([pls_PSI.bootres.contrast_ci[:,0,1], pls_PSI.bootres.contrast_ci[:,1,1]])

y_loadings_m_PSI['ci_l_p'] = y_loadings_m_PSI['weight'] - y_loadings_m_PSI['ci_l']
y_loadings_m_PSI['ci_u_p'] = y_loadings_m_PSI['weight'] - y_loadings_m_PSI['ci_u']
y_loadings_m_PSI['err'] = (y_loadings_m_PSI['ci_u'] - y_loadings_m_PSI['ci_l'])/2

In [ ]:
y_loadings_m_PSI

In [ ]:
#g = sns.catplot(x="time", y="weight", kind="bar", col="salvar_id", ci=None, aspect=1.2, hue="tratamiento",data=y_loadings_m_TAB)

g = sns.catplot(x="time", y="weight", kind="bar", col="salvar_id", ci=None, aspect=1.2, data=y_loadings_m_TAB)

#g.map_dataframe(errplot,  [-0.26,1,1.26,2,2.26], "weight", "err")
g.map_dataframe(errplot,  [0,1,2], "weight", "err")
plt.suptitle('PLS Mean centered TAB')
g.set_xlabels('')


In [ ]:
#g = sns.catplot(x="time", y="weight", kind="bar", col="salvar_id", ci=None, aspect=1.2, hue="tratamiento",data=y_loadings_m_EQF)
#g.map_dataframe(errplot, [-0.26,1,1.26,2,2.26], "weight", "err")

g = sns.catplot(x="time", y="weight", kind="bar", col="salvar_id", ci=None, aspect=1.2,data=y_loadings_m_EQF)
g.map_dataframe(errplot, [0,1,2], "weight", "err")

plt.suptitle('PLS Mean centered EQF')
g.set_xlabels('')


In [ ]:
g = sns.catplot(x="time", y="weight", kind="bar", col="salvar_id", ci=None, aspect=1.2, hue="enfermedad",data=y_loadings_m_PSI)
g.map_dataframe(errplot, [-0.26,1,1.26,2,2.26], "weight", "err")

#g = sns.catplot(x="time", y="weight", kind="bar", col="salvar_id", ci=None, aspect=1.2,data=y_loadings_m_PSI)
#g.map_dataframe(errplot, [0,1,2], "weight", "err")

plt.suptitle('PLS Mean centered EQF+TAB')
g.set_xlabels('')


In [ ]:
pls_EQF.permres.pvals

In [ ]:
pls_EQF.varexp

In [ ]:
pls_TAB.permres.pvals

In [ ]:
pls_TAB.varexp

In [ ]:
pls_PSI.permres.pvals

In [ ]:
pls_PSI.varexp

In [ ]:
print(pls_PSI.bootres.x_weights_normed.min(), pls_PSI.bootres.x_weights_normed.max())

Gráficos de dispersión

In [ ]:
score_EQF = pd.DataFrame()
score_EQF['subject'] = all_data_sorted_EQF['subject']
score_EQF['grupo'] = all_data_sorted_EQF['grupo']
score_EQF['subject'] = all_data_sorted_EQF['subject']
score_EQF['project'] = all_data_sorted_EQF['project']
score_EQF['treatment'] = all_data_sorted_EQF['treatment']
score_EQF['psc'] = all_data_sorted_EQF['psc'].where(pd.notnull(all_data_sorted_EQF['psc']), -1).astype(int)
score_EQF['Gravedad'] = all_data_sorted_EQF['Gravedad']
score_EQF['x_score_1']=pls_EQF.x_scores[:,0]
score_EQF['y_score_1']=pls_EQF.y_scores[:,0]
score_EQF['x_score_2']=pls_EQF.x_scores[:,1]
score_EQF['y_score_2']=pls_EQF.y_scores[:,1]

score_TAB = pd.DataFrame()
score_TAB['subject'] = all_data_sorted_TAB['subject']
score_TAB['grupo'] = all_data_sorted_TAB['grupo']
score_TAB['subject'] = all_data_sorted_TAB['subject']
score_TAB['project'] = all_data_sorted_TAB['project']
score_TAB['treatment'] = all_data_sorted_TAB['treatment']
score_TAB['psc'] = all_data_sorted_TAB['psc'].where(pd.notnull(all_data_sorted_TAB['psc']), -1).astype(int)
score_TAB['Gravedad'] = all_data_sorted_TAB['Gravedad']
score_TAB['x_score_1']=pls_TAB.x_scores[:,0]
score_TAB['y_score_1']=pls_TAB.y_scores[:,0]
score_TAB['x_score_2']=pls_TAB.x_scores[:,1]
score_TAB['y_score_2']=pls_TAB.y_scores[:,1]

In [ ]:
score_PSI = pd.DataFrame()
score_PSI['subject'] = all_data_sorted_PSI['subject']
score_PSI['grupo'] = all_data_sorted_PSI['grupo']
score_PSI['grupo-p'] = all_data_sorted_PSI['grupo-p']
score_PSI['edad'] = all_data_sorted_PSI['edad']
score_PSI['project'] = all_data_sorted_PSI['project']
score_PSI['treatment'] = all_data_sorted_PSI['treatment']
score_PSI['psc'] = all_data_sorted_PSI['psc'].where(pd.notnull(all_data_sorted_PSI['psc']), -1).astype(int)
score_PSI['Gravedad'] = all_data_sorted_PSI['Gravedad']
score_PSI['x_score_1']=pls_PSI.x_scores[:,0]
score_PSI['y_score_1']=pls_PSI.y_scores[:,0]
score_PSI['x_score_2']=pls_PSI.x_scores[:,1]
score_PSI['y_score_2']=pls_PSI.y_scores[:,1]

In [ ]:
score_PSI

In [ ]:
#color = {0: 'w', 1: 'k', 100: 'r'}
color = {0: 'w', 1: 'k', -1: 'r'}
#color = {'controles':'w','prisma1': 'w','prisma3':'k'}
le = LabelEncoder()
subject_n = pd.factorize(score_TAB.apply(lambda x: 'CTRL' if 'CTRL' in x['subject'] else x['subject'], axis=1))[0]
score_TAB['subject_color'] = le.fit_transform(subject_n)
score_TAB['edge_color'] = score_TAB.apply(lambda x: color[x['psc']], axis=1)
#score_TAB['line_width'] = score_TAB.apply(lambda x: line[x['psc']], axis=1)

lee = LabelEncoder()
subject_n = pd.factorize(score_EQF.apply(lambda x: 'CTRL' if 'CTRL' in x['subject'] else x['subject'], axis=1))[0]
score_EQF['subject_color'] = lee.fit_transform(subject_n)
score_EQF['edge_color'] = score_EQF.apply(lambda x: color[x['psc']], axis=1)
#score_TAB['line_width'] = score_TAB.apply(lambda x: line[x['psc']], axis=1)

In [ ]:
color = {0: 'w', 1: 'k', -1: 'r'}
lep = LabelEncoder()
subject_n = pd.factorize(score_PSI.apply(lambda x: 'CTRL' if 'CTRL' in x['subject'] else x['subject'], axis=1))[0]
score_PSI['subject_color'] = lep.fit_transform(subject_n)
score_PSI['edge_color'] = score_PSI.apply(lambda x: color[x['psc']], axis=1)
#score_TAB['line_width'] = score_TAB.apply(lambda x: line[x['psc']], axis=1)

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score_1", y="y_score_1",
                  hue="subject_color", style="project", size='treatment',sizes=[16,64,236],
                  palette='tab20',data=score_EQF,edgecolor=score_EQF["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[61:]
selected_labels = current_labels[61:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE VS Y SCORE LV 1 EQF")
plt.show()

In [ ]:
# Gráfico variable latente 1 vs variable latente 2
sns.set(rc={'figure.figsize':(11,8)})
        
g=sns.scatterplot(x="x_score_1", y="x_score_2",
                  hue="subject_color", style="project", size='treatment',sizes=[16,128,236],legend=True,
                  palette='tab20',data=score_EQF,edgecolor=score_EQF["edge_color"],linewidth=1)


current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[61:]
selected_labels = current_labels[61:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS X SCORE LV2 EQF")
plt.show()

In [ ]:
# Gráfico variable latente 1 vs gravedad
score_EQF_na = score_EQF.dropna(subset=['Gravedad'])
g=sns.scatterplot(x="x_score_1", y="Gravedad",
                 hue="subject_color", style="project", size='treatment',sizes=[16,128,236],
                palette='tab20',data=score_EQF_na,edgecolor=score_EQF_na["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[60:]
selected_labels = current_labels[60:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS Gravedad EQF")
plt.show()

In [ ]:
g=sns.scatterplot(x="x_score_1", y="y_score_1",
                 hue="subject_color", style="project", size='treatment',sizes=[16,64,236],
                palette='tab20',data=score_TAB,edgecolor=score_TAB["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[83:]
selected_labels = current_labels[83:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE VS Y SCORE LV 1 TAB")
plt.show()

In [ ]:
#sns.set(rc={'figure.figsize':(11,8)})
g= sns.scatterplot(x="x_score_1", y="x_score_2",
                hue="subject_color", style="project", size='treatment',sizes=[16,128,236],legend=True,
                palette='tab20',data=score_TAB,edgecolor=score_TAB["edge_color"],linewidth=1)

current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[83:]
selected_labels = current_labels[83:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS X SCORE LV2 TAB")
plt.show()


In [ ]:
score_TAB_na = score_TAB.dropna(subset=['Gravedad'])
markers = {0.0: "X", 1.0: "o",2.0:"s"}
g=sns.scatterplot(x="x_score_1", y="Gravedad",
                  hue="subject_color", style="project", size='treatment',sizes=[16,128,236],
                  palette='tab20',data=score_TAB,edgecolor=score_TAB["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[83:]
selected_labels = current_labels[83:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS Gravedad TAB")
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score_1", y="y_score_1",
                  hue="subject_color", style="project", size='treatment',sizes=[16,64,236],
                  palette='tab20',data=score_PSI,edgecolor=score_PSI["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[147:]
selected_labels = current_labels[147:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE VS Y SCORE LV 1 EQF+TAB")
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score_1", y="y_score_1",
                  hue="grupo", style="project", size='treatment',sizes=[16,64,236],
                  palette='tab20',data=score_PSI,edgecolor=score_PSI["edge_color"],linewidth=1)
#current_handles, current_labels = g.get_legend_handles_labels()
#selected_handles = current_handles[146:]
#selected_labels = current_labels[146:]
#plt.legend(selected_handles,selected_labels)
plt.title("X SCORE VS Y SCORE LV 1 EQF+TAB")
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score_1", y="x_score_2",
                 hue="subject_color", style="project", size='treatment',sizes=[16,128,236],legend=True,
                palette='tab20',data=score_PSI,edgecolor=score_PSI["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[147:]
selected_labels = current_labels[147:]
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS X SCORE LV2 EQF+TAB")
plt.show()

In [ ]:
sizes = {'CTR': 128, 'EQF':236 ,'TAB':236}
score_PSI['grupo-size'] = score_PSI.apply(lambda x: sizes[x['grupo']], axis=1)
#score_TAB['line_width'] = score_TAB.apply(lambda x: line[x['psc']], axis=1)
score_PSI

In [ ]:
g=sns.scatterplot(x="x_score_1", y="x_score_2", size=score_PSI['grupo-size'],
                  hue="grupo", style="project",legend=True,
                  data=score_PSI,edgecolor=score_PSI["edge_color"],linewidth=1)

current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[0:4]
selected_handles.extend(current_handles[7:11])
selected_labels = current_labels[0:4]
selected_labels.extend(current_labels[7:11])
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS X SCORE LV2 EQF+TAB")
plt.show()

In [ ]:
score_PSI_na = score_PSI.dropna(subset=['Gravedad'])
#g=sns.scatterplot(x="x_score_1", y="Gravedad",
#                 hue="subject_color", style="project", size='treatment',sizes=[16,128,236],
#                palette='tab20',data=score_PSI_na,edgecolor=score_PSI_na["edge_color"],linewidth=1)
g=sns.scatterplot(x="x_score_1", y="Gravedad", size=score_PSI['grupo-size'],
                  hue="grupo", style="project",legend=True,
                  data=score_PSI,edgecolor=score_PSI["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[0:4]
selected_handles.extend(current_handles[7:11])
selected_labels = current_labels[0:4]
selected_labels.extend(current_labels[7:11])
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS Gravedad EQF+TAB")
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score_1", y="psc", size=score_PSI['grupo-size'],
                  hue="grupo", style="project",legend=True,
                  data=score_PSI,linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[0:4]
selected_handles.extend(current_handles[7:11])
selected_labels = current_labels[0:4]
selected_labels.extend(current_labels[7:11])
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS Psicosis EQF+TAB")
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(11,8)})
g=sns.scatterplot(x="x_score_1", y="edad", size=score_PSI['grupo-size'],
                  hue="grupo", style="project",legend=True,
                  data=score_PSI,edgecolor=score_PSI["edge_color"],linewidth=1)
current_handles, current_labels = g.get_legend_handles_labels()
selected_handles = current_handles[0:4]
selected_handles.extend(current_handles[7:11])
selected_labels = current_labels[0:4]
selected_labels.extend(current_labels[7:11])
plt.legend(selected_handles,selected_labels)
plt.title("X SCORE LV1 VS Edad EQF+TAB")
plt.show()

## X Weights

In [ ]:
yeo_pal = {'Vis': (0.47058823529411764, 0.07058823529411765, 0.5254901960784314),
 'SomMot': (0.27450980392156865, 0.5098039215686274, 0.7058823529411765),
 'DorsAttn': (0.0, 0.4627450980392157, 0.054901960784313725),
 'SalVentAttn': (0.7686274509803922, 0.22745098039215686, 0.9803921568627451),
 'Limbic': (0.8627450980392157, 0.9725490196078431, 0.6431372549019608),
 'Cont': (0.9019607843137255, 0.5803921568627451, 0.13333333333333333),
 'Default': (0.803921568627451, 0.24313725490196078, 0.3058823529411765),
 'None': (0.7, 0.7, 0.7, 0.5),
 'TempPar': 'blue'}

In [ ]:
EQF_data = all_data_sorted_EQF.iloc[:,3:-6] # Selecciona sólo los datos de volúmenes 
networks7_order = pd.read_csv('Schaefer2018_400Parcels_17Networks_order.txt',header=None,delimiter=' ')

sch_x_sch_weights_EQF = pls_EQF.bootres.x_weights_normed[50:,0] # Selección variables corticales
df_x_weights_sch_EQF = pd.DataFrame(sch_x_sch_weights_EQF, columns=["weight"])
df_x_weights_sch_EQF.index = networks7_order[1]
df_x_weights_sch_EQF["network"] = df_x_weights_sch_EQF.index.str.split('_').str.get(2)

df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('Vis'), 'network'] = 'Vis'
df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('SomMot'), 'network'] = 'SomMot'
df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('DorsAttn'), 'network'] = 'DorsAttn'
df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('SalVentAttn'), 'network'] = 'SalVentAttn'
df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('Limbic'), 'network'] = 'Limbic'
df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('Cont'), 'network'] = 'Cont'
df_x_weights_sch_EQF.loc[df_x_weights_sch_EQF.network.str.contains('Default'), 'network'] = 'Default'

th = 3 
df_x_weights_sch_EQF["weight_th"] =  (abs(df_x_weights_sch_EQF["weight"])>= th)*1


fig, axes = plt.subplots(1, 2, sharex=False, figsize=(15,5))
fig.suptitle('X weights cortical volumes EQF')

order_g = df_x_weights_sch_EQF.groupby(["network"]).mean().sort_values(by='weight').index
g = sns.barplot(x="network", y="weight", ax=axes[0], palette=yeo_pal,
                order=order_g, data=df_x_weights_sch_EQF)

g.set(ylabel='weight intensity')
g.tick_params(labelrotation=60)
order_h = df_x_weights_sch_EQF.groupby(["network"]).sum().sort_values(by='weight_th').index
h = sns.barplot(x="network", y="weight_th", ci=None, ax=axes[1], estimator=sum, palette=yeo_pal,
                order=order_h, data=df_x_weights_sch_EQF)
h.set(ylabel='weight th count')
h.tick_params(labelrotation=60)


In [ ]:
sch_x_v_weights_EQF = pls_EQF.bootres.x_weights_normed[:50,0]
df_x_weights_v_EQF = pd.DataFrame(sch_x_v_weights_EQF, columns=["weight"])
df_x_weights_v_EQF['volume'] = list(EQF_data.iloc[:,:50].columns)
#df_x_weights_v_EQF['category'] = ['Lateral-Ventricle','Lateral-Ventricle','Cerebellum','Cerebellum','Thalamus','Caudate','Putamen','Pallidum','3rd-Ventricle','4th-Ventricle','Brain-Stem','Hippocampus','Amygdala','CSF','Accumbens-area','VentralDC','vessel','choroid-plexus','Lateral-Ventricle','Lateral-Ventricle','Cerebellum','Cerebellum','Thalamus','Caudate','Putamen','Pallidum','Hippocampus','Amygdala','Accumbens-area','VentralDC','vessel','choroid-plexus','Optic-Chiasm','CC','CC','CC','CC','CC']
th = 3
df_x_weights_v_EQF["weight_th"] =  (abs(df_x_weights_v_EQF["weight"])>= th)*1

fig, axes = plt.subplots(1, 2, sharex=True, figsize=(15,5))
fig.suptitle('X weights subcortical volumes EQF')

#g=sns.barplot(x="category", y="weight", ci=None, ax=axes[0],
#            data=df_x_weights_v_EQF)
g=sns.barplot(x="volume", y="weight", ci=None, ax=axes[0],
              data=df_x_weights_v_EQF)
g.set(ylabel='weight intensity')
g.tick_params(labelrotation=60)
h = sns.barplot(x="volume", y="weight_th", ci=None, ax=axes[1], estimator=sum,
            data=df_x_weights_v_EQF)
h.set(ylabel='weight th count')
h.tick_params(labelrotation=60)

In [ ]:
schaefer_image = 'Schaefer2018_400_17N_MNI152_2mm.nii'
sch_masker=input_data.NiftiLabelsMasker(schaefer_image)
sch_values=sch_masker.fit()
th = 3
values_brain_weight_vol = sch_masker.inverse_transform(sch_x_sch_weights_EQF[np.newaxis,:])
plotting.view_img(values_brain_weight_vol, threshold=th, title='X weights EQF', symmetric_cmap=False)

## TAB

In [ ]:
TAB_data = all_data_sorted_TAB.iloc[:,3:-6]
networks7_order = pd.read_csv('Schaefer2018_400Parcels_17Networks_order.txt',header=None,delimiter='\t')
sch_x_sch_weights_TAB = pls_TAB.bootres.x_weights_normed[50:,0]
df_x_weights_sch_TAB = pd.DataFrame(sch_x_sch_weights_TAB, columns=["weight"])
df_x_weights_sch_TAB.index = networks7_order[1]
df_x_weights_sch_TAB["network"] = df_x_weights_sch_TAB.index.str.split('_').str.get(2)

df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('Vis'), 'network'] = 'Vis'
df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('SomMot'), 'network'] = 'SomMot'
df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('DorsAttn'), 'network'] = 'DorsAttn'
df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('SalVentAttn'), 'network'] = 'SalVentAttn'
df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('Limbic'), 'network'] = 'Limbic'
df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('Cont'), 'network'] = 'Cont'
df_x_weights_sch_TAB.loc[df_x_weights_sch_TAB.network.str.contains('Default'), 'network'] = 'Default'

th = 3
df_x_weights_sch_TAB["weight_th"] =  (abs(df_x_weights_sch_TAB["weight"])>= th)*1


fig, axes = plt.subplots(1, 2, sharex=False, figsize=(15,5))
fig.suptitle('X weights cortical volumes TAB')

order_g = df_x_weights_sch_TAB.groupby(["network"]).mean().sort_values(by='weight').index
g = sns.barplot(x="network", y="weight", ax=axes[0], order= order_g,palette=yeo_pal,
            data=df_x_weights_sch_TAB)
g.set(ylabel='weight intensity')
g.tick_params(labelrotation=60)
order_h = df_x_weights_sch_TAB.groupby(["network"]).sum().sort_values(by='weight_th').index
h = sns.barplot(x="network", y="weight_th", ci=None, ax=axes[1], estimator=sum,order= order_h,palette=yeo_pal,
            data=df_x_weights_sch_TAB)
h.set(ylabel='weight th count')
h.tick_params(labelrotation=60)

In [ ]:
sch_x_v_weights_TAB = pls_TAB.bootres.x_weights_normed[:50,0]
df_x_weights_v_TAB = pd.DataFrame(sch_x_v_weights_TAB, columns=["weight"])
df_x_weights_v_TAB['volume'] = list(TAB_data.iloc[:,:50].columns)
#df_x_weights_v_TAB['category'] = ['Lateral-Ventricle','Lateral-Ventricle','Cerebellum','Cerebellum','Thalamus','Caudate','Putamen','Pallidum','3rd-Ventricle','4th-Ventricle','Brain-Stem','Hippocampus','Amygdala','CSF','Accumbens-area','VentralDC','vessel','choroid-plexus','Lateral-Ventricle','Lateral-Ventricle','Cerebellum','Cerebellum','Thalamus','Caudate','Putamen','Pallidum','Hippocampus','Amygdala','Accumbens-area','VentralDC','vessel','choroid-plexus','Optic-Chiasm','CC','CC','CC','CC','CC']
th = 3
df_x_weights_v_TAB["weight_th"] =  (abs(df_x_weights_v_TAB["weight"])>= th)*1

fig, axes = plt.subplots(1, 2, sharex=True, figsize=(15,5))
fig.suptitle('X weights subcortical volumes TAB')

g=sns.barplot(x="volume", y="weight", ci=None, ax=axes[0],
            data=df_x_weights_v_TAB)
g.set(ylabel='weight intensity')
g.tick_params(labelrotation=60)
h = sns.barplot(x="volume", y="weight_th", ci=None, ax=axes[1], estimator=sum,
            data=df_x_weights_v_TAB)
h.set(ylabel='weight th count')
h.tick_params(labelrotation=60)

In [ ]:
schaefer_image = 'Schaefer2018_400_17N_MNI152_2mm.nii'
sch_masker=input_data.NiftiLabelsMasker(schaefer_image)
sch_values=sch_masker.fit()
#th = max(sch_x_sch_weights_TAB)*0.6
th=3
values_brain_weight_vol = sch_masker.inverse_transform(sch_x_sch_weights_TAB[np.newaxis,:])
plotting.view_img(values_brain_weight_vol, threshold=th, title='X weights TAB', symmetric_cmap=False)

EQF+TAB

In [ ]:
PSI_data = all_data_sorted_PSI.iloc[:,3:-7]
networks7_order = pd.read_csv('Schaefer2018_400Parcels_17Networks_order.txt',header=None,delimiter='\t')
sch_x_sch_weights_PSI = pls_PSI.bootres.x_weights_normed[50:,0]
df_x_weights_sch_PSI = pd.DataFrame(sch_x_sch_weights_PSI, columns=["weight"])
df_x_weights_sch_PSI.index = networks7_order[1]
df_x_weights_sch_PSI["network"] = df_x_weights_sch_PSI.index.str.split('_').str.get(2)

df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('Vis'), 'network'] = 'Vis'
df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('SomMot'), 'network'] = 'SomMot'
df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('DorsAttn'), 'network'] = 'DorsAttn'
df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('SalVentAttn'), 'network'] = 'SalVentAttn'
df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('Limbic'), 'network'] = 'Limbic'
df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('Cont'), 'network'] = 'Cont'
df_x_weights_sch_PSI.loc[df_x_weights_sch_PSI.network.str.contains('Default'), 'network'] = 'Default'

th = 3
df_x_weights_sch_PSI["weight_th"] =  (abs(df_x_weights_sch_PSI["weight"])>= th)*1


fig, axes = plt.subplots(1, 2, sharex=False, figsize=(15,5))
fig.suptitle('X weights cortical volumes EQF+TAB')

order_g = df_x_weights_sch_PSI.groupby(["network"]).mean().sort_values(by='weight').index
g = sns.barplot(x="network", y="weight", ax=axes[0], order= order_g,palette=yeo_pal,
            data=df_x_weights_sch_PSI)
g.set(ylabel='weight intensity')
g.tick_params(labelrotation=60)
order_h = df_x_weights_sch_PSI.groupby(["network"]).sum().sort_values(by='weight_th').index
h = sns.barplot(x="network", y="weight_th", ci=None, ax=axes[1], estimator=sum,order= order_h,palette=yeo_pal,
            data=df_x_weights_sch_PSI)
h.set(ylabel='weight th count')
h.tick_params(labelrotation=60)
# Incluir ROIs en una tabla

In [ ]:
df_x_weights_sch_PSI = df_x_weights_sch_PSI[df_x_weights_sch_PSI['weight_th']>0]
df_x_weights_sch_PSI.reset_index(inplace=True)
df_x_weights_sch_PSI.columns = ['structure', 'weight','network','weight_th']
y = df_x_weights_sch_PSI[['structure','network']]
y = y.sort_values(by=['network'])
y

In [ ]:
sch_x_v_weights_PSI = pls_PSI.bootres.x_weights_normed[:49,0]
df_x_weights_v_PSI = pd.DataFrame(sch_x_v_weights_PSI, columns=["weight"])
df_x_weights_v_PSI['volume'] = list(PSI_data.iloc[:,:49].columns)
#df_x_weights_v_TAB['category'] = ['Lateral-Ventricle','Lateral-Ventricle','Cerebellum','Cerebellum','Thalamus','Caudate','Putamen','Pallidum','3rd-Ventricle','4th-Ventricle','Brain-Stem','Hippocampus','Amygdala','CSF','Accumbens-area','VentralDC','vessel','choroid-plexus','Lateral-Ventricle','Lateral-Ventricle','Cerebellum','Cerebellum','Thalamus','Caudate','Putamen','Pallidum','Hippocampus','Amygdala','Accumbens-area','VentralDC','vessel','choroid-plexus','Optic-Chiasm','CC','CC','CC','CC','CC']
th = 3
df_x_weights_v_PSI["weight_th"] =  (abs(df_x_weights_v_PSI["weight"])>= th)*1

fig, axes = plt.subplots(1, 2, sharex=True, figsize=(15,5))
fig.suptitle('X weights subcortical volumes EQF+TAB')

g=sns.barplot(x="volume", y="weight", ci=None, ax=axes[0],
            data=df_x_weights_v_PSI)
g.set(ylabel='weight intensity')
g.tick_params(labelrotation=60)
h = sns.barplot(x="volume", y="weight_th", ci=None, ax=axes[1], estimator=sum,
            data=df_x_weights_v_PSI)
h.set(ylabel='weight th count')
h.tick_params(labelrotation=60)

In [ ]:
schaefer_image = 'Schaefer2018_400_17N_MNI152_2mm.nii' 
sch_masker=input_data.NiftiLabelsMasker(schaefer_image)
sch_values=sch_masker.fit()
th=3
values_brain_weight_vol = sch_masker.inverse_transform(sch_x_sch_weights_PSI[np.newaxis,:])
plotting.view_img(values_brain_weight_vol, threshold=th, title='X weights EQF+TAB', symmetric_cmap=False)

In [ ]:
# Ploting stat map
#(barrido axial), que se puedan superponer los bordes de las redes (plot contours)
schaefer_image = 'Schaefer2018_400_17N_MNI152_2mm.nii'
display = plotting.plot_stat_map(values_brain_weight_vol, display_mode='mosaic', cut_coords=(3,3,3), title='X weights EQF+TAB', 
                       threshold=th, dim = 0, symmetric_cbar=False, black_bg=True,
                       colorbar=True)
display.add_contours(schaefer_image, cmap='Paired')


In [ ]:
schaefer_image = 'Schaefer2018_400_17N_MNI152_2mm.nii'
sch_masker=input_data.NiftiLabelsMasker(schaefer_image)
sch_values=sch_masker.fit()
plotting.plot_roi(schaefer_image, view_type='contours', title="", cmap='Paired')
plotting.show()